# Data Exploration

I began by familiarizing myself with the Reuters dataset. If you don't have the data, use nltk.download() from a Python module.

In [10]:
import nltk

nltk.corpus.reuters

<CategorizedPlaintextCorpusReader in u'/home/austin/nltk_data/corpora/reuters.zip/reuters/'>

In [27]:
len(nltk.corpus.reuters.fileids())

10788

In [11]:
nltk.corpus.reuters.fileids()[:5]

['test/14826', 'test/14828', 'test/14829', 'test/14832', 'test/14833']

In [13]:
from nltk.corpus import reuters
reuters.fileids()[:5]

['test/14826', 'test/14828', 'test/14829', 'test/14832', 'test/14833']


Each document has "categories" which I can weigh differently later.

In [32]:
test_article = reuters.fileids()[6]
reuters.categories(test_article)

[u'coffee', u'lumber', u'palm-oil', u'rubber', u'veg-oil']

The beginning of each article has a title in all capital letters.

In [33]:
reuters.words(test_article)[:6]

[u'INDONESIAN', u'COMMODITY', u'EXCHANGE', u'MAY', u'EXPAND', u'The']

I'm going to preserve these tokens, and weigh them stronger relative to the corpus.

Next it makes sense for me to create a Document object. It needs to have a dictionary of the category tokens, dictionary of the title tokens, 

In [44]:
corpus = []

for file_id in reuters.fileids():
    document = {}
    
    for category in reuters.categories(file_id):
        category = category.encode('utf-8')
        if category in document:
            document[category] += 10
        else:
            document[category] = 10
    
    for word in reuters.words(file_id):
        word = word.encode('utf-8')
        
    
    corpus.append(document)
    
print corpus[6]

{'rubber': 10, 'coffee': 10, 'lumber': 10, 'veg-oil': 10, 'palm-oil': 10}
